In [2]:
from src.data import read_raw_data, preprocess_data

data = read_raw_data()
data = preprocess_data(data)

[Pipeline]  (step 1 of 3) Processing functiontransformer-1, total=   0.0s
[Pipeline]  (step 2 of 3) Processing monthspeciestraptransformer, total=   0.0s
[Pipeline]  (step 3 of 3) Processing functiontransformer-2, total=   0.0s


In [3]:
from src.features import add_num_multirows, SpeciesEncoder, TrapFeatureExtractor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer


multirows_counter = FunctionTransformer(add_num_multirows)
species_oh_encoder = SpeciesEncoder()
trap_feat_extractor = TrapFeatureExtractor()

feature_pipeline = make_pipeline(
    multirows_counter,
    species_oh_encoder,
    trap_feat_extractor
)

data['train'] = feature_pipeline.fit_transform(data['train'])
data['test'] = feature_pipeline.transform(data['test'])


In [5]:
from sklearn.preprocessing import KBinsDiscretizer


lat_enc = KBinsDiscretizer(n_bins=3, encode='ordinal')

data['train']['lat_bin'] = lat_enc.fit_transform(data['train'][['Latitude']])
data['test']['lat_bin'] = lat_enc.transform(data['test'][['Latitude']])

long_enc = KBinsDiscretizer(n_bins=3, encode='ordinal')

data['train']['long_bin'] = long_enc.fit_transform(data['train'][['Longitude']])
data['test']['long_bin'] = long_enc.transform(data['test'][['Longitude']])

In [6]:
data['train']

,Date,Trap,Latitude,Longitude,NumMosquitos,WnvPresent,Month,Year,Week,Dayofyear,Dayofweek,NumRows,Species_CULEX PIPIENS,Species_CULEX PIPIENS/RESTUANS,Species_CULEX RESTUANS,trap_max_mos,trap_wnv_proba,lat_bin,long_bin
0,2007-07-02,T002,41.954690,-87.800991,3,0,7,2007,27,183,0,1,0.0,1.0,0.0,582,0.244898,2.0,0.0
1,2007-07-02,T002,41.954690,-87.800991,2,0,7,2007,27,183,0,1,0.0,0.0,1.0,582,0.244898,2.0,0.0
2,2007-07-02,T015,41.974089,-87.824812,3,0,7,2007,27,183,0,1,0.0,1.0,0.0,39,0.307692,2.0,0.0
3,2007-07-02,T045,41.921600,-87.666455,2,0,7,2007,27,183,0,1,0.0,1.0,0.0,175,0.181818,1.0,1.0
4,2007-07-02,T045,41.921600,-87.666455,2,0,7,2007,27,183,0,1,0.0,0.0,1.0,175,0.181818,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6012,2013-09-26,T228,41.951866,-87.725057,3,0,9,2013,39,269,3,1,0.0,1.0,0.0,123,0.200000,2.0,1.0
6013,2013-09-26,T035,41.763733,-87.742302,6,1,9,2013,39,269,3,1,0.0,1.0,0.0,31,0.085714,1.0,1.0
6014,2013-09-26,T231,41.987280,-87.666066,5,0,9,2013,39,269,3,1,0.0,1.0,0.0,150,0.200000,2.0,1.0
6015,2013-09-26,T233,42.009876,-87.807277,5,0,9,2013,39,269,3,1,0.0,1.0,0.0,200,0.190476,2.0,0.0


In [7]:
train_columns = ['Week', 'Dayofyear', 'Dayofweek',
       'trap_max_mos', 'trap_wnv_proba', 'lat_bin', 'long_bin',
       'Species_CULEX PIPIENS', 'Species_CULEX PIPIENS/RESTUANS',
       'Species_CULEX RESTUANS', 'WnvPresent']

test_columns = ['Id', 'Week', 'Dayofyear', 'Dayofweek',
       'trap_max_mos', 'trap_wnv_proba', 'lat_bin', 'long_bin',
       'Species_CULEX PIPIENS', 'Species_CULEX PIPIENS/RESTUANS',
       'Species_CULEX RESTUANS']

In [9]:
X_train = data['train'][train_columns].copy()
y_train = X_train.pop('WnvPresent')

In [17]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate, RepeatedStratifiedKFold


scaler = StandardScaler()
clf = LogisticRegression()

steps = [
    ('scale', scaler),
    ('classifier', clf)
]
pipeline = Pipeline(steps=steps)

cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3)

results_cv = cross_validate(pipeline, X_train, y_train, cv=cv, scoring='roc_auc')

print(results_cv['test_score'].mean())

0.6708914104842586
